In [22]:
import csv
import numpy as np
import tensorflow as tf
from scipy.stats import norm
from scipy.stats import uniform

## CNN vs. DNN

## Generate Data
(Skip this if you want to use already generated data) 

Make sure that the parameters below where the same when training the networks

In [23]:
num_strikes = 12
num_maturities = 12
num_sample_patches = 1
num_data_points = num_strikes * num_maturities * num_sample_patches


#initial values
S0 = 1.0
V0 = 0.2
r = 0.15

num_model_params = 3
num_contract_params = 2

contract_bounds = np.array([[0.7*S0,1.3*S0],[1,10]]) #bounds for K,T
model_bounds = np.array([[0,5],[0,1],[0,1]]) #bounds for alpha,beta,rho, make sure alpha>0, beta,rho \in [0,1]

# make sure does distances are the same in output grid approach and image input for cnn
maturities_distance = (contract_bounds[1,1]-contract_bounds[1,0])/(2*num_maturities) 
strikes_distance = (contract_bounds[0,1]-contract_bounds[0,0])/(2*num_strikes)

alpha_true = 1.0
beta_true = 0.5
rho_true = 0.2

In [41]:
def corr_brownian_motion(n, T, dim, rho):
    dt = T/n

    dW1 = norm.rvs(size=(dim,n+1) , scale=np.sqrt(dt))
    dW2 = rho * dW1 + np.sqrt(1 - np.power(rho ,2)) * norm.rvs(size=(dim,n+1) , scale=np.sqrt(dt))
        
    W1 = np.cumsum(dW1, axis=-1)
    W2 = np.cumsum(dW2, axis=-1)
 
    return W1,W2

def euler_maruyama(mu,sigma,T,x0,W):
    dim = W.shape[0]
    n = W.shape[1]-1
    Y = np.zeros((dim,n+1))
    dt = T/n
    sqrt_dt = np.sqrt(dt)
    for l in range(dim):
        Y[l,0] = x0
        for i in range(n):
            Y[l,i+1] = Y[l,i] + np.multiply(mu(Y[l,i],l,i),dt) + sigma(Y[l,i],l,i)*sqrt_dt*(W[l,i+1]-W[l,i])
    
    return Y

def sabr(alpha,beta,T,W,Z,V0,S0):
    assert(beta>0)

    def mu2(V,i,k):
        return 0.0
    
    def sigma2(V,i,k):
        return np.multiply(alpha,V)
    
    V = euler_maruyama(mu2,sigma2,T,V0,Z)
    
    def mu1(S,i,k):
        return 0.0
    
    def sigma1(S,i,k):
        return np.multiply(V[i,k],np.power(np.maximum(0.0,S),beta))
    
    S = euler_maruyama(mu1,sigma1,T,S0,W)
    
    return S,V

def price_pred(alpha,beta,rho,n,dim,T,K,V0,S0):
    W,Z = corr_brownian_motion(n,T,dim,rho)
    S,V = sabr(alpha,beta,T,W,Z,V0,S0)
    S_T = S[:,n]
    P = np.exp(-r*T) * np.mean(np.maximum(S_T-K,np.zeros(dim)))
    
    return P


In [42]:
data = np.zeros((num_contract_params+1,num_data_points))
for i in range(num_sample_patches):
    for j in range(num_maturities):
        strike_low = contract_bounds[0,0]
        T = maturity_low + j * maturities_distance
        for k in range(num_strikes):
            K = strike_low + k * strikes_distance
            n = 100
            dim =50
            data[0,i*num_maturities*num_strikes+j*num_strikes+k] = T
            data[1,i*num_maturities*num_strikes+j*num_strikes+k] = K
            data[2,i*num_maturities*num_strikes+j*num_strikes+k] = price_pred(alpha_true,beta_true,rho_true,n,dim,T,K,V0,S0)
            
            

## Load Models

In [70]:
tf.reset_default_graph()
imported_meta = tf.train.import_meta_graph("../SABR_Experiments/models/sabr_cnn.meta")
graph = tf.get_default_graph()

X_test = np.zeros((1,num_maturities,num_strikes,1))
for i in range(num_maturities):
    for j in range(num_strikes):
        X_test[0,i,j,0] = data[2,i*num_strikes+j]

with graph.as_default() as default_graph:                          
    #print([op.name for op in graph.get_operations()])
    #X = tf.get_variable('Placeholder')
    #tf.get_operation_by_name('outputs')
    with tf.Session() as sess:
        imported_meta.restore(sess, '../SABR_Experiments/models/checkpoint')
        params_method_1 = default_graph.run(outputs,feed_dict={X: X_test})

print(params_method_1)


INFO:tensorflow:Restoring parameters from ../SABR_Experiments/models/checkpoint


DataLossError: Unable to open table file ../SABR_Experiments/models/checkpoint: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "/Users/robinvogtland/opt/anaconda3/envs/slt-ce/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/robinvogtland/opt/anaconda3/envs/slt-ce/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/robinvogtland/opt/anaconda3/envs/slt-ce/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/User